# AMUSE: coupling codes

In [ ]:
%matplotlib inline
import numpy
numpy.random.seed(11)
from amuse.lab import *
from amuse.support.console import set_printing_strategy
set_printing_strategy(
    "custom",
    preferred_units=[units.MSun, units.parsec, units.Myr, units.kms],
    precision=6, prefix="", separator=" [", suffix="]",
)
converter = nbody_system.nbody_to_si(1 | units.parsec, 1000 | units.MSun)
test_sphere = new_plummer_model(1000, converter)
test_sphere.mass = new_salpeter_mass_distribution(1000, mass_min=0.3 | units.MSun)
def new_gravity(particles):
    gravity = ph4(converter, number_of_workers=1)
    gravity.parameters.epsilon_squared = (0.01 | units.parsec)**2
    gravity.particles.add_particles(particles)
    gravity_to_model = gravity.particles.new_channel_to(particles)
    return gravity, gravity_to_model

One of the main advantages of AMUSE is that we can use it to combine codes. This way we can address a problem that involves multiple scales and/or physical processes.

The perhaps simplest case is combining gravity with a stellar evolution code, where the coupling is one-way only. A more complex case is when we want to couple a hydrodynamical code with a high-precision gravity solver, e.g. for simulating a star cluster embedded in a molecular cloud. In this case, we need both systems to interact via gravity, and a special method is required.

In AMUSE, the Bridge module is designed for this. In Bridge, we can add multiple systems and specify which other systems each of these needs to interact with. By default, this interaction is then executed via a leapfrog-type integration, but higher order methods are available (if supported by the underlaying systems).

In [ ]:
from amuse.couple.bridge import Bridge
def new_sph(particles):
    sph = Fi(converter, mode="openmp")
    # Note that most SPH codes support DM particles as well, so 
    # we need to specifically add particles to "gas_particles"
    sph.gas_particles.add_particles(particles)
    sph.parameters.timestep = 0.05 | units.Myr
    sph_to_model = sph.gas_particles.new_channel_to(particles)
    return sph, sph_to_model
test_gas = new_plummer_gas_model(1000, converter)

To show the interaction, I will add one obvious particle to follow

In [ ]:
leftmost_particle = test_sphere.select(
    lambda x:
    x == test_sphere.x.min(),
    ["x"]
)
leftmost_particle.mass = 100 | units.MSun
leftmost_particle.position = [-2, 0, 0] | units.parsec
leftmost_particle.velocity = [0, 1.5, 0] | units.parsec * units.Myr**-1

In [ ]:
sph, sph_to_model = new_sph(test_gas)
gravity, gravity_to_model = new_gravity(test_sphere)

system = Bridge(timestep=2 * sph.parameters.timestep)

system.add_system(
    gravity,  # System to be influenced
    partners=[sph],  # List of partner codes that will affect the system
)
system.add_system(
    sph,
    partners=[gravity],
)

In [ ]:
from plotting_class import plot_hydro_and_stars
plot_hydro_and_stars(system.model_time, sph, test_sphere,)

In [ ]:
time = system.model_time
tend = 2.0 | units.Myr
while time <= tend:
    time += 0.2 | units.Myr
    print("Evolving to %s" % time)
    system.evolve_model(time)
    gravity_to_model.copy()
    plot_hydro_and_stars(system.model_time, sph, test_sphere, title="%s" % time)

In [ ]:
gravity.stop()
sph.stop()